# Merge multiple well data files(多个井数据文件合并)

Prevent multiple well data files from being stored in one CSV file(将多口井数据文件防止到一个csv文件中)

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import math
# import seaborn as sns
# import pylab as pl

# from mpl_toolkits.axes_grid1 import make_axes_locatable
# from scipy import interpolate
from pandas import set_option
set_option("display.max_rows", 15)

In [2]:
# 使用GPU训练时候，打开下面注释
import os
# os.environ['CUDA_VISIBLE_DEVICES']='0'

# Set the types of curves to be included in the merged results(设定要合并结果中包含的曲线种类)

In [3]:
# 定义要处理的常规测井资料
# usual_log_names = ["AC","CNL","DEN","GR","RD","RS"]
# usual_log_names = ["DT","CNL","DEN","GR","RD","RS",'DTXX']
usual_log_names = ["CAL","SP","GR","CNL","DT","DEN","MSFL","RS","RD"]

# 定义是否要使用常规测井资料，True | False
use_usual_log = True
use_custom_depth = False
merge_mode = "Up_Down"  # default:Left-Right  | Up_Down

# Loading Data读入数据

In [5]:

result_data_path = '../data/exp_curve_reconstract/exp_2/train/QSK/'
# filename 取值例如：'TT1-3085m-5078m.csv','J404-3394m-4257m.csv'
file_path_list = []
all_result_csvfile_name = ""#'TT1-4500m-5070m_R_0.1524.csv'
# all_result_save_path = 'csv_results/汇总/'
all_result_save_path = '../data/exp_curve_reconstract/exp_2/train/QSK/'
print(all_result_csvfile_name)
for parent, dirnames, filenames in os.walk(result_data_path,  followlinks=True):
    # print(filenames)
    for filename in filenames:
        file_path = os.path.join(parent, filename)
        file_path_list.append(file_path)
    print(str(parent).rstrip("/").split("/"))
        # print(os.path.dirname(parent))
        # all_result_csvfile_name = parent.split("/")[-1] + '_all_result.csv'
    all_result_csvfile_name = str(parent).rstrip("/").split("/")[-1] + '_all.csv'
        # print('文件名：%s' % filename)
        # print('文件完整路径：%s\n' % file_path)
print('结果文件名：%s\n' % all_result_csvfile_name)
print('结果文件完整路径：%s\n' % all_result_save_path)
#原文链接：https://blog.csdn.net/huplion/article/details/79537334


['facies_data', 'train', '地质4口卡奔图-1-58']
结果文件名：地质4口卡奔图-1-58_all.csv

结果文件完整路径：facies_data/train/



In [6]:
file_path_list

['facies_data/train/地质4口卡奔图-1-58/古页-1_facies_vectors_0.1.csv',
 'facies_data/train/地质4口卡奔图-1-58/英斜-58井_facies_vectors_0.1.csv']

In [7]:
depth_data = pd.read_csv(file_path_list[0],engine='python',encoding='GBK')
depth_data

,DEPTH,CAL,SP,GR,CNL,DT,DEN,MSFL,RS,RD,Well Name,Facies
0,2304.5,10.8180,102.90,128.0,18.60,96.0,2.4600,2.170,10.020,9.440,1,3
1,2304.6,10.7556,102.66,128.0,19.00,96.8,2.4600,2.034,9.548,9.040,1,3
2,2304.7,10.7130,102.42,128.6,19.58,97.0,2.4582,2.006,9.118,8.664,1,3
3,2304.8,10.7442,102.22,129.0,20.30,97.0,2.4546,2.086,8.778,8.364,1,3
4,2304.9,10.8426,102.10,128.8,21.06,97.0,2.4488,2.242,8.546,8.160,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2816,2586.1,8.7800,98.60,152.2,17.56,93.8,2.6162,5.766,5.492,4.482,1,2
2817,2586.2,8.7800,98.54,149.0,17.02,91.2,2.6242,7.284,5.734,4.746,1,2
2818,2586.3,8.7796,98.50,145.0,16.38,89.2,2.6346,8.520,5.994,5.014,1,2
2819,2586.4,8.7790,98.46,141.2,15.80,88.0,2.6448,8.676,6.204,5.222,1,2


In [8]:
# if use_custom_depth == False:
#     pd_data = None     
# else:
#     pd_data = pd.DataFrame(depth_data,columns=["DEPTH"])
# pd_data

In [9]:
pd_data = None   

In [10]:
for item_file in file_path_list:
    item_data = pd.read_csv(item_file,engine='python',encoding='GBK')
    # pd_data1 = item_data.iloc[:,1:]
    pd_data1 = item_data
    if merge_mode == "Up_Down":
        # 选择合并方式
        pd_data =pd.concat([pd_data,pd_data1],axis=0,ignore_index=True)
    else:
        pd_data =pd.concat([pd_data,pd_data1],axis=1)

In [11]:
pd_data

,DEPTH,CAL,SP,GR,CNL,DT,DEN,MSFL,RS,RD,Well Name,Facies
0,2304.5,10.8180,102.9000,128.0000,18.6000,96.0000,2.4600,2.1700,10.0200,9.4400,1,3
1,2304.6,10.7556,102.6600,128.0000,19.0000,96.8000,2.4600,2.0340,9.5480,9.0400,1,3
2,2304.7,10.7130,102.4200,128.6000,19.5800,97.0000,2.4582,2.0060,9.1180,8.6640,1,3
3,2304.8,10.7442,102.2200,129.0000,20.3000,97.0000,2.4546,2.0860,8.7780,8.3640,1,3
4,2304.9,10.8426,102.1000,128.8000,21.0600,97.0000,2.4488,2.2420,8.5460,8.1600,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
3908,2121.0,8.6860,167.7170,138.9920,19.5630,91.9960,2.5790,6.6650,5.4110,4.8130,X58,3
3909,2121.1,8.7100,167.8194,133.5352,19.7814,89.8688,2.5622,6.2690,5.7942,5.1658,X58,6
3910,2121.2,8.7220,167.7712,126.5028,19.7904,87.2976,2.5448,7.2152,6.4504,5.7964,X58,6
3911,2121.3,8.7248,167.6516,120.2804,19.4012,84.6108,2.5292,10.1568,7.3772,6.7212,X58,6


# Writing data to csv(结果写入csv)

In [12]:
pd_data.to_csv(all_result_save_path + all_result_csvfile_name,index = False,encoding='GBK',header=True)

In [13]:
print("Finished!!")
print(str(all_result_save_path) + str(all_result_csvfile_name))

Finished!!
facies_data/train/地质4口卡奔图-1-58_all.csv
